# Obtain valid set of reactions 
Required Python code to load pertinent modules.

In [ ]:
import thermoengine as thermo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
%matplotlib notebook

In [ ]:
modelDB = thermo.model.Database()

In [ ]:
phase_symbols = ['Grt', 'Cpx', 'Ol']
svd_props_eq = thermo.chem.calc_reaction_svd(phase_symbols, modelDB=modelDB)

In [ ]:
print(svd_props_eq.keys())

In [ ]:
svd_props_eq['all_endmember_name']

In [ ]:
phs_symbols_all = list(modelDB.phase_attributes.keys())
exclude = ['O2','S2', 'H2', 'KlsS', 'LctS', 'Mll', 'NphS', ]
[phs_symbols_all.remove(sym) for sym in exclude];
print(phs_symbols_all)
svd_props_full = thermo.chem.calc_reaction_svd(phs_symbols_all, modelDB=modelDB)

In [ ]:
len(svd_props_full['all_endmember_name'])

In [ ]:
svd_props_full.keys()

In [ ]:
phs_symbols_all = ['Fsp','Grt', 'Cpx', 'Ol',  'Qz']
print(phs_symbols_all)
svd_props_full = thermo.chem.calc_reaction_svd(phs_symbols_all, modelDB=modelDB)

In [ ]:
[print(phs, ' : ', endmem) for phs,endmem 
 in zip(svd_props_eq['all_phase_name'], 
        svd_props_eq['all_endmember_name'])];

In [ ]:
svd_props_full['all_phase_symbol']
svd_props_full['all_endmember_id']

In [ ]:
[print(phs, ' : ', endmem) for phs,endmem 
 in zip(svd_props_full['all_phase_name'], 
        svd_props_full['all_endmember_name'])];

In [ ]:
svd_props_full['all_mol_oxide_comp'].shape

In [ ]:
svd_props_eq['all_mol_oxide_comp'].shape

In [ ]:
svd_eq = svd_props_eq['rxn_svd']
svd_full = svd_props_full['rxn_svd']

In [ ]:
rxn0 = svd_props_eq['rxn_svd']
phs_sym0 = svd_props_eq['all_phase_symbol']
endmem_id0 = svd_props_eq['all_endmember_id']
phs_sym_full = svd_props_full['all_phase_symbol']
endmem_id_full = svd_props_full['all_endmember_id']
print(len(phs_sym_full))
print(len(endmem_id_full))

print(phs_sym0, endmem_id0)
print(phs_sym_full, endmem_id_full)

In [ ]:
def expand_rxn_matrix(rxn0, phs_sym0, endmem_id0, 
                      phs_sym_full, endmem_id_full):
    
    phs_sym_full = np.array(phs_sym_full)
    endmem_id_full = np.array(endmem_id_full)
    
    Nendmem = len(endmem_id_full)
    Nrxn = rxn0.shape[0]
    rxn_full = np.zeros((Nrxn, Nendmem))
    
    ind_full = []
    for iphs_sym, iendmem_id in zip(phs_sym0, endmem_id0):
        # print(iphs_sym, iendmem_id)
        ind = np.where((phs_sym_full==iphs_sym)&
                       (endmem_id_full==iendmem_id))[0][0]
        ind_full.append(ind)
        
    ind_full = np.array(ind_full)
    
    for i, irxn0 in enumerate(rxn0):
        rxn_full[i, ind_full] = irxn0
        
    phs_absent = np.ones(Nendmem)
    phs_absent[ind_full] = 0
    
    return rxn_full, phs_absent
        

# rxn0 = np.round(rxn0, decimals=2)

rxn0_full, phs_absent = expand_rxn_matrix(rxn0, phs_sym0, endmem_id0, 
                      phs_sym_full, endmem_id_full)

# svd_props_full['all_phase_symbol']

# svd_props_full['all_endmember_id']

In [ ]:
phs_absent

In [ ]:
rxn_full = svd_props_full['rxn_svd']
rxn_full.shape

In [ ]:
rxn_resid = rxn_full.copy()
for irxn0 in rxn0_full:
    rxn_resid -= np.dot(np.dot(rxn_resid, irxn0)[:,np.newaxis], 
                  irxn0[np.newaxis, :])

In [ ]:

plt.figure()
plt.imshow(rxn0_full)
plt.title('rxn0')


plt.figure()
plt.imshow(rxn_full)
plt.title('rxn_full')

plt.figure()
plt.imshow(rxn_resid)
plt.title('rxn_resid')


In [ ]:
u, s, vh = np.linalg.svd(rxn_resid)

In [ ]:
rxn_rank = vh[0:3]
rxn_rank

In [ ]:
plt.figure()
plt.imshow(rxn_rank, cmap='coolwarm')
plt.colorbar()

In [ ]:
from scipy.optimize import linprog

In [ ]:
rxn_both = np.vstack((rxn_rank,-rxn_rank))


In [ ]:
phs_absent_ind = np.where(phs_absent)[0]
phs_absent_ind

for absent_ind in phs_absent_ind:
    print(rxn_both[:,absent_ind])

In [ ]:
cost = np.array([1,1,1,1,1,-1,-1,-1,-1,-1])

In [ ]:
rxn_both.shape[0]

In [ ]:
Nrxn = 5
A_eq = np.vstack([np.ones(2*Nrxn), rxn_both[:,0], rxn_both[:,1],
                  rxn_both[:,2]])
b_eq = np.array([1, 0, 0, 0])
b_eq
A_eq.shape
print(b_eq)
print(A_eq)

In [ ]:
out = linprog(cost, A_eq=A_eq, b_eq=b_eq, bounds=(None, None))
out.x


In [ ]:
np.round(np.dot(rxn_both.T, out.x), decimals=2)

In [ ]:
out

In [ ]:
from scipy.optimize import linprog

In [ ]:
rxn_both = np.vstack((rxn0,-rxn0))
rxn_both[:,0].size


In [ ]:
rxn_both[:,0]

In [ ]:
cost = np.array([1,1,1,1,1,-1,-1,-1,-1,-1])

In [ ]:
rxn_both.shape[0]

In [ ]:
Nrxn = 5
A_eq = np.vstack([np.ones(2*Nrxn), rxn_both[:,0], rxn_both[:,1],
                  rxn_both[:,2]])
b_eq = np.array([1, 0, 0, 0])
b_eq
A_eq.shape
print(b_eq)
print(A_eq)

In [ ]:
out = linprog(cost, A_eq=A_eq, b_eq=b_eq, bounds=(None, None))
out.x


In [ ]:
np.round(np.dot(rxn_both.T, out.x), decimals=2)

In [ ]:
out

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
print(svd_eq.shape)

plt.figure()
plt.imshow(svd_eq, cmap='coolwarm')
plt.colorbar()

# Quick aside on calculating molar endmember compositions

In [ ]:
phs = modelDB.get_phase('Grt')
phs0 = modelDB.get_phase('Ky')

print('phs0 = ', phs0.endmember_names, ' --> ', phs0.endmember_formulas)
print('phs = ', phs.endmember_names, ' --> ', phs.endmember_formulas)

In [ ]:
mol_oxide_alm = chem.format_mol_oxide_comp({'FeO':3, 'Al2O3':1, 'SiO2':3})
mol_oxide_grs = chem.format_mol_oxide_comp({'CaO':3, 'Al2O3':1, 'SiO2':3})
mol_oxide_pyr = chem.format_mol_oxide_comp({'MgO':3, 'Al2O3':1, 'SiO2':3})

mol_oxide_mix =  chem.format_mol_oxide_comp({'FeO':.9, 'CaO':1, 'MgO':1.1, 
                                             'Al2O3':1, 'SiO2':3})

In [ ]:
print('Endmember inferences:')
print('alm: ', phs.calc_endmember_comp(mol_oxide_alm)[0])
print('grs: ', phs.calc_endmember_comp(mol_oxide_grs)[0])
print('pyr: ', phs.calc_endmember_comp(mol_oxide_pyr)[0])
print('mix: ', phs.calc_endmember_comp(mol_oxide_mix)[0])

## We already have an update to calculating endmember comp in master (that we need to pull in)
* To use it, just add the keyword argument to calc_endmember_comp:
    * method='intrinsic'

In [ ]:
rxn_data=pd.read_excel(data_dir+'grt_bearing_expts.xls',sheetname=None)

In [ ]:
rxn_data['phase_symbols']

### Find relevant phases according to data

In [ ]:
#TK delete this?
rel_phases = rxn_data['phase_symbols']['phase_symbol'].tolist()

modelDB.phase_symbols
mask_phases = [iphase in modelDB.phase_symbols for iphase in rel_phases]
rel_phases[mask_phases]

In [ ]:
relevant_phases = chem.get_relevant_phases(rxn_data)

In [ ]:
relevant_phases

In [ ]:
phase_names = phases.PurePhase

In [ ]:
phase_names

In [ ]:
props = modelDB.phase_attributes['props']

In [ ]:
endmember_names = phases.PurePhase(relevant_phases)
#phases_present=rxn_data['phase_symbols']
#relevant_phases = rxn_data['phase_symbols']['phase_symbol'].tolist()

### Retrieve singular value decomposition matrix for all reactions:
➜ output is a matrix of valid, linearly independent reactions that minimize variance  and maximizes orthogonality of each vector against another.

➜ Input a tolerance, defined as...

In [ ]:
rxn_svd, rxn_svd_props = chem.calc_reaction_svd(relevant_phases, TOLsvd=1e-4)

In [ ]:
rxn_svd

In [ ]:
rxn_svd_props['all_endmember_id']

In [ ]:
rxn_svd_props['all_endmember_name']

In [ ]:
Nbasis=len(rxn_svd)

### Get set of filtered reactions
➜ output is matrix of valid, lineraly independent reactions filtered based on given input variables described below

➜ input

    rxn_svd: matrix from above

    Ndraw: 

    ortho_scale: ratio that dictates level of rxn complexity (orthogonality/simplicity)

    Nbasis:

    TOL: rxn_complexity
 
note: the following operation make take ~10 minutes

In [ ]:
wtcoefs, costs, rxn_coefs, wtcoefs_ortho = chem.get_rxns(rxn_svd, Ndraw=2, ortho_scale=15, Nbasis=Nbasis, TOL=1e-10)

In [ ]:
rxns_file = '/Users/jennaadams/Documents/projects/ThermoEngine/Notebooks/Solutions/data/rxns_file.pkl'
with open(rxns_file, 'wb') as handle:
    pickle.dump(rxn_coefs, handle)
    
with open (rxns_file, 'rb') as handle:
    rxns_file_open = pickle.load(handle)
    
rxns_file_open

## Reaction Visiualization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

### Visualizing rxn_svd

In [ ]:
rxn_max = np.max(rxn_svd)

plt.figure()
plt.imshow(rxn_svd,cmap='seismic')
plt.clim(-rxn_max,+rxn_max)
plt.colorbar()

plt.xlabel('Endmember ID#')
plt.ylabel('Balanced Rxn ID#')

### All reactions after filtering rxn_svd

In [ ]:
plt.figure()
plt.imshow(rxn_coefs,cmap='seismic')
plt.clim(-rxn_max, +rxn_max)
plt.colorbar()

plt.xlabel('Endmember ID#')
plt.ylabel('Balanced Rxn ID#')

### Reaction correlation

In [ ]:
ortho_project =np.dot(rxn_coefs, rxn_coefs.T) 

plt.figure()
plt.imshow(ortho_project, cmap='seismic')
plt.colorbar()
plt.clim(-1,1)

plt.xlabel('Basic Rxn ID#')
plt.ylabel('Basic Rxn ID#')
plt.title('Basic Rxn Correlation')

### Subset of new reactions 

In [ ]:
plt.figure()
plt.plot(rxn_coefs[::10].T, '-')
plt.xlabel('')
plt.ylabel('rxn')
plt.legend(['0','10','20','30'])

### Create final reaction matrix

In [ ]:
unique_phase_symbols=chem.get_phase_symbols(rxn_data)
rxn_svd_props = chem.calc_reaction_svd(unique_phase_symbols, TOLsvd=1e-4)
rxn_svd = rxn_svd_props['rxn_svd']
Nbasis=len(rxn_svd)
rxn_endmember_name = rxn_svd_props['all_endmember_name']
rxn_phase_symbols = rxn_svd_props['all_phase_symbol']



In [ ]:
rxn_svd_props.keys()

In [ ]:
rxn_svd_props['all_phase_ind']

In [ ]:
wtcoefs, costs, rxn_coefs, wtcoefs_ortho = chem.get_rxns(rxn_svd, Ndraw=2, ortho_scale=15, Nbasis=Nbasis, TOL=1e-10)

In [ ]:
(np.place(rxn_coefs, abs(rxn_coefs)< 1e-2, 0))

In [ ]:
endmember_ids = rxn_svd_props['all_endmember_id']

In [ ]:
len(rxn_phase_symbols)

In [ ]:
len(endmember_ids)

In [ ]:
len(rxn_endmember_name)

In [ ]:
rxns = []
for irxn_coefs in rxn_coefs:
    irxn = modelDB.get_rxn(rxn_phase_symbols, endmember_ids, irxn_coefs)
    
    rxns.append(irxn)    

In [ ]:
phases = []

for irxn in rxns:
    phases.extend(irxn.phases)
    
phases = np.unique(phases)

In [ ]:
rxn_phase_symbols

In [ ]:
phases

In [ ]:
rxn_coefs[0]

In [ ]:
Rxn.reactant_phases



In [ ]:
phase_info = phases.Rxn(rxn_phase_symbols, endmember_ids, irxn_coefs)